Usei isso para conseguir trainML e testML

In [6]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('./ML20Data/ratings.csv')
df

,user_id,book_id,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
...,...,...,...,...
20000258,138493,68954,4.5,1258126920
20000259,138493,69526,4.5,1259865108
20000260,138493,69644,3.0,1260209457
20000261,138493,70286,5.0,1258126944


In [7]:
df = df.drop(columns=['timestamp'])
df = df.drop_duplicates(subset=['book_id', 'user_id'], keep='first')
df

,user_id,book_id,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5
...,...,...,...
20000258,138493,68954,4.5
20000259,138493,69526,4.5
20000260,138493,69644,3.0
20000261,138493,70286,5.0


In [8]:
user_counts = df['user_id'].value_counts()
users_to_keep = user_counts[user_counts > 1].index

df = df[df['user_id'].isin(users_to_keep)]
df

,user_id,book_id,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5
...,...,...,...
20000258,138493,68954,4.5
20000259,138493,69526,4.5
20000260,138493,69644,3.0
20000261,138493,70286,5.0


In [9]:
nan_count = df.isna().sum().sum()
print(f"Total NaN values in the DataFrame: {nan_count}")

Total NaN values in the DataFrame: 0


In [10]:
grouped = df.groupby('user_id')

train_df = pd.DataFrame(columns=df.columns)
test_df = pd.DataFrame(columns=df.columns)

for user_id, group in tqdm(grouped, desc="Processing users"):
    test_sample = group.sample(n=1, random_state=42)
    train_sample = group.drop(test_sample.index)
    test_df = pd.concat([test_df, test_sample], ignore_index=True)
    train_df = pd.concat([train_df, train_sample], ignore_index=True)

Processing users:   0%|          | 0/138493 [00:00<?, ?it/s]/tmp/ipykernel_1265227/2541592586.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  test_df = pd.concat([test_df, test_sample], ignore_index=True)
/tmp/ipykernel_1265227/2541592586.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  train_df = pd.concat([train_df, train_sample], ignore_index=True)
Processing users: 100%|██████████| 138493/138493 [9:26:05<00:00,  4.08it/s]  


In [11]:
test_value_counts = test_df['user_id'].value_counts()
test_unique_counts = test_df['user_id'].nunique()

print(test_value_counts)
print(test_unique_counts)

user_id
1         1
92325     1
92339     1
92338     1
92337     1
         ..
46160     1
46159     1
46158     1
46157     1
138493    1
Name: count, Length: 138493, dtype: int64
138493


In [12]:
train_df.to_csv('trainML.csv', index=False)
test_df.to_csv('testML.csv', index=False)